In [ ]:
!pip install ultralytics wandb roboflow -q

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=user_secrets.get_secret("roboflow_api_key"))
project = rf.workspace("kelvin-snkig").project("ppe-akpdb")
version = project.version(2)
dataset = version.download("yolov11")        

In [ ]:
from ultralytics import settings

settings.update({"wandb": True})

print(settings['wandb'])

In [ ]:
import wandb

wandb.login(key=user_secrets.get_secret("wandb"))

In [ ]:
from ultralytics import YOLO

# Catat konfigurasi train yang digunakan untuk di save ke dalam wandb
config = {
    "data": f"{dataset.location}/data.yaml",
    "pretrained_model": "yolo11x.pt", # ganti model yang dilatih disini
    "imgsz": 640,
    "batch_size": 16,
    "optimizer": "SGD",
    "epochs": 50
}


wandb.init(
    project="PPE Detection Model",
    config=config,
    name=f'{config["pretrained_model"]}-{config["epochs"]}-{config["batch_size"]}-{config["imgsz"]}-{config["optimizer"]}'
)

model = YOLO(config["pretrained_model"])

model.train(
    data= config["data"],
    epochs= config["epochs"],
    batch= config["batch_size"],        
    imgsz= config["imgsz"],                 
    optimizer= config["optimizer"],
    device=[0, 1] # parameter ini diperlukan karena error out of memory yang disebabkan oleh batch size yang besar sehingga 1 gpu tak cukup untuk melatihnya
)

wandb.finish()